<a href="https://colab.research.google.com/github/thainhf/new-taggerbot-training-model/blob/main/Clean_data_taggerbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polar
!pip install pythainlp
!pip install emoji

In [63]:
import gspread as gs
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from google.auth import default
import nltk
import polar as pl

<module 'polar.polar' from '/usr/local/lib/python3.8/dist-packages/polar/polar.py'>

In [112]:
df_defin = pd.read_excel('/content/drive/MyDrive/Taggerbot/Tag_Definition.xlsx', header=0)
df_train_IPST = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_IPST_20Expert.xlsx', header=0)
df_train_Lump100 = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_Lumplymat_100Doc_10Expert.xlsx', header=0)
df_train_Lumpkk = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_Lumplymat_KK_Conf.xlsx', header=0)
df_train_Qinfo = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_Qinfo2_15Expert.xlsx', header=0)
df_train_school = pd.read_excel('/content/drive/MyDrive/Taggerbot/Training_School_Management.xlsx', header=0)


"""df_defin = pl.DataFrame(df_defin)
df_train_IPST = pl.DataFrame(df_train_IPST)
df_train_Lump100 = pl.DataFrame(df_train_Lump100)
df_train_Lumpkk = pl.DataFrame(df_train_Lumpkk)
df_train_Qinfo = pl.DataFrame(df_train_Qinfo)
df_train_school = pl.DataFrame(df_train_school)"""

FileNotFoundError: ignored

# Cleaning Data

In [113]:
df_train_IPST = df_train_IPST.rename(columns={'name.1':'tag'})
df_train_Qinfo = df_train_Qinfo.rename(columns={'name.1':'tag'})

In [114]:
df_train_Lump100 = df_train_Lump100.rename(columns={'Tag':'tag', 'Paragraph_Text':'content'})
df_train_Lumpkk = df_train_Lumpkk.rename(columns={'Tag':'tag', 'Paragraph_Text':'content'})
df_train_school = df_train_school.rename(columns={'Tag':'tag', 'Paragraph_Text':'content'})

In [115]:
df_train_main = pd.concat([df_train_IPST[['content','tag']], df_train_Lump100[['content','tag']], df_train_Lumpkk[['content','tag']],
                           df_train_Qinfo[['content','tag']], df_train_school[['content','tag']]], axis=0)
df_train_main['tag'] = df_train_main['tag'].str.lower()

df_train_main['tag'] = df_train_main['tag'].replace('ict  literacy', 'ict literacy')
df_train_main['tag'] = df_train_main['tag'].replace('information lieteracy skill', 'information literacy skill')
df_train_main['tag'] = df_train_main['tag'].replace('media literacy', 'media literacy skills')
df_train_main['tag'] = df_train_main['tag'].replace('productivity', 'productivity and accountability')
df_train_main['tag'] = df_train_main['tag'].replace('critical thinking', 'critical thinking and problem solving')

df_train_main

,content,tag
0,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,analyse
1,จากสถานการณ์ที่กำหนดให้ นักเรียนตอบคำถามลงในใบ...,analyse
2,นักเรียนแต่ละคนภายในกลุ่มนำข้อมูลที่ได้ศึกษาค้...,analyse
3,การเคลื่อนที่ของเม็ดโฟมผูกด้วยด้ายเล็ก ที่มีคว...,analyse
4,สึนามิแตกต่างจากคลื่นธรรมดาอย่างไร,analyse
...,...,...
1624,มากไปกว่านั้น เด็กๆ ยังคิดต่อได้ว่า ควรมีสาระ...,critical thinking and problem solving
1625,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,information literacy skill
1626,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,ict literacy
1627,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,media literacy skills


In [116]:
df_train_main[df_train_main.duplicated()]

,content,tag
34,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,analyse
47,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,analyse
56,ออกแบบแผนอพยพหนีภัยสึนามิและการสร้างเส้นทางกา...,analyse
88,2) นักเรียนคิดว่า สึนามิแตกต่างจากค...,analyse
89,นักเรียนแต่ละกลุ่มสร้างแผนผังแนวคิดการทำงาน พร...,analyse
...,...,...
5092,ให้นักเรียนวัดความยาวฝากระป๋อง และความยาวเส้นผ...,critical thinking and problem solving
5094,ให้นักเรียนวัดความยาวถังน้ำ และความยาวเส้นผ่าน...,critical thinking and problem solving
5100,ร่วมกิจกรรมและปฏิบัติตนเป็นสมาชิกที่ดีในกลุ่...,collaboration skill
5105,ร่วมกิจกรรมและปฏิบัติตนเป็นสมาชิกที่ดีในกลุ่...,collaboration skill


In [117]:
print(len(set(df_train_main['tag'])), 'skills')
df_train_main['tag'].value_counts()

20 skills


critical thinking and problem solving    10084
communication skill                       5034
collaboration skill                       3867
initiative and self-direction             3127
flexibility and adaptability              1955
social and cross-cultural skills          1574
creativity and innovation                 1563
information literacy skill                1294
productivity and accountability           1286
understanding                             1040
leadership and responsibity                907
analyse                                    694
media literacy skills                      467
ict literacy                               459
evaluate                                   342
remember                                   302
apply                                      252
compassion                                 176
decision skill                             152
accountability                             137
Name: tag, dtype: int64

In [120]:
df_train_main['content'] = df_train_main['content'].str.replace('[\\<>\;\[\]\:\-*\*\$\,\.\?!"\'\^\()\&\/\{\}\|\£\«\»\’\“\”\•\•\■\™\~]\s*', '', regex=True)
df_train_main = df_train_main.reset_index(drop=True)
df_train_main

,content,tag
0,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,analyse
1,จากสถานการณ์ที่กำหนดให้ นักเรียนตอบคำถามลงในใบ...,analyse
2,นักเรียนแต่ละคนภายในกลุ่มนำข้อมูลที่ได้ศึกษาค้...,analyse
3,การเคลื่อนที่ของเม็ดโฟมผูกด้วยด้ายเล็ก ที่มีคว...,analyse
4,สึนามิแตกต่างจากคลื่นธรรมดาอย่างไร,analyse
...,...,...
34707,มากไปกว่านั้น เด็กๆ ยังคิดต่อได้ว่า ควรมีสาระ...,critical thinking and problem solving
34708,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,information literacy skill
34709,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,ict literacy
34710,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,media literacy skills


In [121]:
df_train_main[df_train_main.duplicated()]

,content,tag
34,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,analyse
47,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,analyse
56,ออกแบบแผนอพยพหนีภัยสึนามิและการสร้างเส้นทางกา...,analyse
88,2นักเรียนคิดว่า สึนามิแตกต่างจากคลื...,analyse
89,นักเรียนแต่ละกลุ่มสร้างแผนผังแนวคิดการทำงาน พร...,analyse
...,...,...
33027,ให้นักเรียนวัดความยาวฝากระป๋อง และความยาวเส้นผ...,critical thinking and problem solving
33029,ให้นักเรียนวัดความยาวถังน้ำ และความยาวเส้นผ่าน...,critical thinking and problem solving
33035,ร่วมกิจกรรมและปฏิบัติตนเป็นสมาชิกที่ดีในกลุ่...,collaboration skill
33040,ร่วมกิจกรรมและปฏิบัติตนเป็นสมาชิกที่ดีในกลุ่...,collaboration skill


In [122]:
df = (df_train_main.groupby(['content'])
      .agg({'tag': lambda x: ",".join(x)})
      .reset_index())
df_contain = df[df['tag'].str.contains(',')]
df_contain

,content,tag
1,ส่วนร่ว...,"collaboration skill,flexibility and adaptability"
5,นอกจากนี้ครูยังจัดที่นั...,"accountability,flexibility and adaptability"
6,ผลที่เกิดขึ้นกับเด็กทำใ...,"communication skill,collaboration skill"
9,การที่ได้นำเป้าหมายประจำสัป...,"accountability,initiative and self-direction"
12,จากเหตุการณ์ครั้งนี้ทำให้รู้ว่...,"communication skill,collaboration skill"
...,...,...
6345,๔งานการ์ตูนมีการบรรยายความรู้แทรก,"critical thinking and problem solving,communic..."
6349,๕งานโฟกัสมีการเปรียบเทียบใบพัดของพัดลมกับเฮลิค...,"critical thinking and problem solving,communic..."
6350,๕นักเรียนเป็นนักสังเกต เข้าหาความรู้ได้เอง,"productivity and accountability,accountability..."
6351,๕ประเด็นที่จะนำไปพัฒนาต่อ คือ การฝึกตั้งประเด็...,"creativity and innovation,initiative and self-..."


In [119]:
df_contain.iloc[0,0]

'                                       ส่วนร่วมในการดูแลรักษาสิ่งแวดล้อม'

# Tokenization

In [149]:
#from pythainlp.ulmfit.utils import get_texts
from pythainlp.tokenize import word_tokenize

In [148]:
df_train_main = df_train_main.assign(token = [word_tokenize(x, engine="newmm") for x in df_train_main['content']])
#df_train_main = df_train_main.drop(columns='token')
#df_train_main.insert(1, 'token', word_tokenize(df_train_main.content.str, engine="newmm"))
df_train_main

,content,tag,token
0,สึนามิในวิดีทัศน์มีลักษณะเหมือนหรือแตกต่างจากค...,analyse,"[สึนามิ, ใน, วิดีทัศน์, มี, ลักษณะ, เหมือน, หร..."
1,จากสถานการณ์ที่กำหนดให้ นักเรียนตอบคำถามลงในใบ...,analyse,"[จาก, สถานการณ์, ที่, กำหนดให้, , นักเรียน, ต..."
2,นักเรียนแต่ละคนภายในกลุ่มนำข้อมูลที่ได้ศึกษาค้...,analyse,"[นักเรียน, แต่ละคน, ภายใน, กลุ่ม, นำ, ข้อมูล, ..."
3,การเคลื่อนที่ของเม็ดโฟมผูกด้วยด้ายเล็ก ที่มีคว...,analyse,"[การ, เคลื่อนที่, ของ, เม็ด, โฟม, ผูก, ด้วย, ด..."
4,สึนามิแตกต่างจากคลื่นธรรมดาอย่างไร,analyse,"[สึนามิ, แตกต่าง, จาก, คลื่น, ธรรมดา, อย่างไร, ]"
...,...,...,...
34707,มากไปกว่านั้น เด็กๆ ยังคิดต่อได้ว่า ควรมีสาระ...,critical thinking and problem solving,"[มาก, ไป, กว่า, นั้น, , เด็ก, ๆ, , ยัง, คิด..."
34708,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,information literacy skill,"[ใน, การเขียน, ประมวล, ความรู้, หน่วย, วิชา, ป..."
34709,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,ict literacy,"[ใน, การเขียน, ประมวล, ความรู้, หน่วย, วิชา, ป..."
34710,ในการเขียนประมวลความรู้หน่วยวิชาประยุกต์ การบ้...,media literacy skills,"[ใน, การเขียน, ประมวล, ความรู้, หน่วย, วิชา, ป..."


In [133]:
word_tokenize(df_train_main.content[0], engine="newmm")

['สึนามิ',
 'ใน',
 'วิดีทัศน์',
 'มี',
 'ลักษณะ',
 'เหมือน',
 'หรือ',
 'แตกต่าง',
 'จาก',
 'คลื่น',
 'ทะเล',
 'ทั่วไป',
 'อย่างไร',
 ' ']

# Modeling